In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.metrics.pairwise import pairwise_kernels
import joblib
from sklearn import metrics

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

In [2]:
#Hard dataset

X, y = make_classification(n_classes=3, n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=2)

In [3]:
#Toy dataset
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [4]:
a, b = make_classification(n_classes=3, n_samples=500, n_features=6, n_informative=5, n_redundant=1, random_state=2)
c, d = make_classification(n_classes=3, n_samples=500, n_features=6, n_informative=3, n_redundant=3, random_state=44)
X = np.concatenate((a,c))
y= np.concatenate((b,d))

In [5]:
#Serious dataset

X, y = make_classification(n_classes=3, n_samples=1000, n_features=6, n_informative=4, n_redundant=2, random_state=2)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [7]:
from random import randint
import random
from joblib import Parallel, delayed

def _validate(metric, bootstrap_method):
    
    if bootstrap_method is not None:
        if bootstrap_method not in ("rejection","randomchoice"):
            raise ValueError(
                'Invalid preset "%s" for bootstrap_method'
                % bootstrap_method
                )
    
    if metric is not None:
        if metric not in ("rbf", "laplacian"):
            raise ValueError(
                'Invalid preset "%s" for kernel metric'
                % metric
                )
            
def _pickDiverseSeed(sim_matrix,seed=None):
    #creating seed for specific training set, choosing by inverse similarity to previous set
    if seed is None:
            return randint(0,len(sim_matrix)-1)
    else:
        #questo è probabilmente il codice più cringe che io abbia mai scritto,
        #ma non riesco a fargli ritornare un int se non specificando l'indice

        sim_n = 1-sim_matrix[seed]

        candidate = np.random.choice(np.arange(len(sim_matrix)), size=1, replace=True,
                                     p=sim_n/np.sum(sim_n))[0]

        return candidate
    
def _pickDiverseSeed2(sim_matrix,seed): #Randomly picks from all used seeds, does not improve much
    l=len(seed)
    if len(seed) == 0:
        return randint(0,len(sim_matrix)-1)
    else:
        
        candidates = np.zeros(l)
        
        for i in range(0,len(seed)):

            sim_n = 1-sim_matrix[seed[i]]

            candidates[i] = np.random.choice(np.arange(len(sim_matrix)), size=1, replace=True, 
                                             p=sim_n/np.sum(sim_n))[0]
        

        return int(np.random.choice(candidates, size=1, replace=True)[0])
    
def _parallel_fit(n_estimators, base_estimator, X, y, i):
       
            model = base_estimator.fit(X[i], y[i])
            model_name = "model"+str(i)+".pkl"
            joblib.dump(model, model_name)
    


In [8]:
def _randomchoice_bootstrap(X, y, similarity_metric, n_estimators, verbose=0):
    #Initializing list of random training sets
    X_sets = []
    y_sets = []
    G = pairwise_kernels(X, metric=similarity_metric)
    
    seed=None
    
    for i in range(0,n_estimators):
        instance_X = np.empty(np.shape(X))
        instance_y = np.empty(np.shape(y))
        
        #creating seed for specific training set
        
        if verbose>0:
            oldseed=seed
        
        seed = _pickDiverseSeed(G,seed)
        
        if verbose>0 and oldseed is not None:
            print("Chose new seed",seed,"based on previous one:",oldseed,"w/ similarity:",G[seed][oldseed])
                  

        instance_X[0] = X[seed]
        instance_y[0] = y[seed]
        
        pool = np.arange(np.shape(X)[0])
        pool_prob = np.zeros(np.shape(X)[0])
        pool_tot = np.sum(G[seed])
        
        
        for i in range(np.shape(X)[0]):
            pool_prob[i] = G[seed][i]/pool_tot
        
        if False:  #adds noise, does not increase accuracy
            for i in range(0,int(np.shape(X)[0]/2)):
                offset = 0.3 #random.uniform(0, 1)
                
                index = randint(0,np.shape(X)[0]-1)
                while pool_prob[index] + offset > 1: 
                    index = randint(0,np.shape(X)[0]-1)
                pool_prob[index] += offset
                
                index = randint(0,np.shape(X)[0])-1
                while pool_prob[index] - offset < 0: 
                    index = randint(0,np.shape(X)[0]-1)
                pool_prob[index] -= offset
                
                
#         print(pool_prob)
        
#         print(G[seed])
        
        pick = np.random.choice(pool, size=(len(pool))-1, replace=True, p=pool_prob)
        
        for i in range(len(pick)):
            instance_X[i+1] = X[pool[i]]
            instance_y[i+1] = y[pool[i]]
        
        X_sets.append(instance_X)
        y_sets.append(instance_y)
    
    return X_sets,y_sets

#Test
X_sets,y_sets = _randomchoice_bootstrap(X_train,y_train,"rbf",n_estimators=50,verbose=0)

In [9]:
from sklearn.metrics import pairwise_distances


def _rejection_bootstrap(X, y, similarity_metric, n_estimators, n_jobs):
    
    #Initializing list of random training sets
    X_sets = []
    y_sets = []
    G = pairwise_kernels(X, metric=similarity_metric)
    
    seed=None

# CURRENTLY RETURNS ERROR IF n_jobs!=1
#     Parallel(n_jobs=n_jobs)(delayed(_parallel_rejection_bootstrap)(G,seed,n_estimators,X,y,X_sets,y_sets) 
#                                   for i in range(0,n_estimators))
    
    for i in range(0,n_estimators):  #for each estimator  
            _parallel_rejection_bootstrap(G,seed,n_estimators,X,y,X_sets,y_sets)   
    
    return X_sets,y_sets

def _parallel_rejection_bootstrap(G,seed,n_estimators,X,y,X_sets,y_sets):
    instance_X = np.empty(np.shape(X))
    instance_y = np.empty(np.shape(y))

    seed = _pickDiverseSeed(G,seed)

    instance_X[0] = X[seed]
    instance_y[0] = y[seed]       

    n_entries = 1

    #populating training set


    if 1-(len(X_sets)+1)/n_estimators>0.8:
        p_thresh = 0.8
    else:
        p_thresh = np.round(1-(len(X_sets)+1)/n_estimators,1)

    #print("populating set", i, "with probability acceptance",p_thresh)


    while n_entries < len(X): #until the pool isn't filled up

        rand = randint(0,len(X)-1)

        if _accept_entry(G[seed,rand],n_entries/(len(X)-1),p_thresh):

            instance_X[n_entries] = X[rand]
            instance_y[n_entries] = y[rand]
            n_entries+=1

    X_sets.append(instance_X)
    y_sets.append(instance_y)

def _accept_entry(similarity_score, p_ratio, prob_threshold = 0.7):
    #p_ratio gradually shifts the decisional weight from similarity_score to randomness as the test set gets filled up.
    #Specifically, at least least a small number of entries are accepted only through randomness (p_ratio=1)
    #this way it's unlikely for the method to come up with sets with only one class (if not impossible, but I need to check)
    
    #It is also possible to modify the prob_threshold in order to get more or less random values, assuring more variance.
    #I think this tweakability could prove useful to adapt the model to specific instances.
    
    #PLEASE NOTE THAT THIS CODE MAKES SENSE WITH NORMALIZED SIMILARITY SCORES
    #NORMALIZATION FOR FUNCTION THAT DO NOT RETURN [0,1] VALUES STILL NEEDS TO BE IMPLEMENTED.
    #It's not like it doesn't work, but prob calculations take the high road
    
    prob = similarity_score*(1-p_ratio)+ random.random()*p_ratio
    if prob > prob_threshold:
        return True
    else:
        return False
    
    
#Test
X_sets, y_sets = _rejection_bootstrap(X_train, y_train,'rbf',50,1)

In [10]:
"""
   
    Parameters
    ----------
    n_estimators : int, default=50
        The number of models to train.
        
    base_estimator : estimator, default=DecisionTreeClassifier()
        The estimator fitted on  each bootstrapped set.     
        
    n_jobs : int, default=4
        Number of parallel jobs during fitting.
        
    similarity_metric : {"rbf", "laplacian", "cosine"}, string, default="rbf"
        The metric used for pairwise_kernels().
        
    bootstrap_method={"rejection", "randomchoice"}, string, default=TODO
        The bootstrap method of choice.
    
    verbose : int, default = 0
        Controls verbosity during fitting and predicting, 0 being none and 3 being the most detailed. 
    
    


"""



class CustomEstimator(BaseEstimator, ClassifierMixin):
    
    def __init__(self, n_estimators=50,
                base_estimator=DecisionTreeClassifier(),
                n_jobs=1,
                bootstrap_method="rejection",
                similarity_metric="rbf",
                verbose = 0):
        
        
        
        self.n_estimators = n_estimators 
        self.base_estimator = base_estimator
        self.n_jobs = n_jobs
        self.bootstrap_method = bootstrap_method
        self.similarity_metric = similarity_metric
        self.verbose = verbose 
        
        _validate(self.similarity_metric, self.bootstrap_method)
        
    def fit(self,X,y):
        
        self.n_classes_= np.max(y)+1
        
        if self.verbose > 0:
            print("Bootstrapping...")
            
        if self.bootstrap_method=="rejection":  
            self.X_sets, self.y_sets = _rejection_bootstrap(X, y, self.similarity_metric, self.n_estimators, self.n_jobs)
        elif self.bootstrap_method=="randomchoice": 
            self.X_sets, self.y_sets = _randomchoice_bootstrap(X, y, self.similarity_metric, self.n_estimators)
        
        
        if self.verbose > 0:
            print("Fitting...")
        
        Parallel(n_jobs=self.n_jobs)(delayed(_parallel_fit)(self.n_estimators, self.base_estimator, self.X_sets, self.y_sets, i) 
                           for i in range(0,self.n_estimators))
        
#         for i in range(self.n_estimators):
#             _parallel_fit(self.n_estimators, self.base_estimator, self.X_sets, self.y_sets, i)
            
        
        self.X_ = X
        self.y_ = y

        if self.verbose > 0:
            print("Done!")
        return self
                
    
    def predict(self, X):
        
        predicted_probabilitiy = self.predict_proba(X)
        return np.argmax(predicted_probabilitiy, axis=1)
    
    def predict_proba(self, X):
            
        if self.verbose > 0:
            print("Predicting...")
            
        out = np.zeros(shape=(len(X),self.n_classes_))
        
        #compute similarity between instance and each training set
        
        if self.verbose > 0:
            print("Computing models similarity to instance...")
        
        Z=[]
        for j in range (0,self.n_estimators):
            Z.append(pairwise_kernels(X, self.X_sets[j], metric=self.similarity_metric))
#             if self.verbose > 1:
#                 print("Similarity matrix for instance to model",j,":",Z[j])
        
        
        
        if self.verbose > 0:
            print("Making predictions with each trained model...")
            
          
        for i in range(0,len(X)): #for each sample to predict
            if self.verbose > 1:  
                print("\n--- EVALUATING SAMPLE",i," ---")
            
            votes = []
            votes_weighted = []
            sim_means = np.zeros(self.n_estimators)
            
            voting_power = np.zeros(self.n_estimators)
            
            for j in range (0,self.n_estimators): #for each estimator trained
               
                #load estimator model
                model_name = "model"+str(j)+".pkl"
                model = joblib.load(model_name)
                
                votes.append(model.predict_proba(X[i].reshape(1, -1))) 
                sim_means[j] = np.mean(Z[j][i])
                
                
            for j in range (0,self.n_estimators):
                     
                voting_power[j] = (100*sim_means[j]/np.sum(sim_means))/100
                votes_weighted.append(votes[j]*voting_power[j])
                
                if self.verbose > 2:
                    print("model ",j,"votes:",np.round(votes[j],4),"having ~",np.round(sim_means[j],4),
                          "similarity (voting power:",np.round(voting_power[j],4),")")
                
            out[i]=(np.sum(votes_weighted,axis=0))
            if self.verbose > 1:
                print("FINAL PREDICTION:",out[i],)
            
        
        if self.verbose > 0:
            print("Done!")

        return out

In [11]:
# Test instance 

clf = CustomEstimator(n_estimators=50, base_estimator=DecisionTreeClassifier(), n_jobs=4,
                             similarity_metric='rbf', bootstrap_method="randomchoice",verbose=1)

%time clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print("Accuracy =",metrics.accuracy_score(y_test, y_pred))

Bootstrapping...
Fitting...
Done!
CPU times: total: 516 ms
Wall time: 3.06 s
Predicting...
Computing models similarity to instance...
Making predictions with each trained model...
Done!
Accuracy = 0.805


In [12]:
#Test parallelizzazione
clf1 = CustomEstimator(n_estimators=50, base_estimator=DecisionTreeClassifier(), n_jobs=1,
                             similarity_metric='rbf', bootstrap_method="rejection")

clf2 = CustomEstimator(n_estimators=50, base_estimator=DecisionTreeClassifier(), n_jobs=4,
                             similarity_metric='rbf', bootstrap_method="rejection")

%time clf1.fit(X_train, y_train)
%time clf2.fit(X_train, y_train)

CPU times: total: 4.2 s
Wall time: 4.25 s
CPU times: total: 5.42 s
Wall time: 5.49 s


CustomEstimator(n_jobs=4)

In [13]:
c1 = DecisionTreeClassifier()
c2 = CustomEstimator(n_estimators=50,base_estimator=c1,verbose=1,n_jobs=2)
c3 = RandomForestClassifier(n_estimators=50, max_leaf_nodes=16, n_jobs=-1)

c1.fit(X_train,y_train)
c2.fit(X_train,y_train)
c3.fit(X_train,y_train)


print(c1.predict_proba(X_test))
print(c2.predict_proba(X_test))
print(c3.predict_proba(X_test))

Bootstrapping...
Fitting...
Done!
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0

In [14]:
#Accuracy comparison w/different parameters
from sklearn.ensemble import VotingClassifier

clf1 = CustomEstimator(n_estimators=50,base_estimator=DecisionTreeClassifier())
clf2 = CustomEstimator(n_estimators=100,base_estimator=DecisionTreeClassifier(),bootstrap_method="randomchoice")
clf3 = CustomEstimator(n_estimators=50,base_estimator=DecisionTreeClassifier(),similarity_metric="laplacian")
clf4 = CustomEstimator(n_estimators=100,base_estimator=DecisionTreeClassifier(),similarity_metric="laplacian",bootstrap_method="randomchoice")

estimators = [("rbf/rej",clf1),("rbf/raC",clf2),("Lap/rej",clf3),("Lap/raC",clf4)]

for i in range(0,len(estimators)):
    estimators[i][1].fit(X_train, y_train)
    y_pred = estimators[i][1].predict(X_test)
    print(estimators[i][0],":",metrics.accuracy_score(y_test, y_pred))
    

rbf/rej : 0.885
rbf/raC : 0.79
Lap/rej : 0.9
Lap/raC : 0.79


In [15]:
# Comparative tests

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

rnd_clf = RandomForestClassifier(n_estimators=50, max_leaf_nodes=16, n_jobs=-1)

svc_clf = SVC(probability=True)

tree_clf = DecisionTreeClassifier(max_depth=16)

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=50, max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True)

custom_clf = CustomEstimator(n_estimators=50,base_estimator=DecisionTreeClassifier())

custom_clf2 = CustomEstimator(n_estimators=50,similarity_metric="laplacian")

custom_clf3 = CustomEstimator(n_estimators=50,similarity_metric="laplacian", bootstrap_method="randomchoice")

print("Accuracy scores:")
for clf in (rnd_clf, svc_clf, tree_clf, bag_clf, custom_clf, custom_clf2, custom_clf3):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,":",metrics.accuracy_score(y_test, y_pred))
    
#     results, names = list(), list()
#     scores = evaluate_model(clf, X, y)
#     results.append(scores)
#     names.append(clf.__class__.__name__)
#     print('%s %.3f (%.3f)' % ("Cross validation score:", np.mean(scores), np.std(scores)))



Accuracy scores:
RandomForestClassifier : 0.815
SVC : 0.865
DecisionTreeClassifier : 0.81
BaggingClassifier : 0.83
CustomEstimator : 0.835
CustomEstimator : 0.865
CustomEstimator : 0.805


In [16]:
from sklearn.metrics.pairwise import check_pairwise_arrays

pairwise_kernels(X, Y=None, metric="cosine")

#[‘additive_chi2’, ‘chi2’, ‘linear’, ‘poly’, ‘polynomial’, ‘rbf’, ‘laplacian’, ‘sigmoid’, ‘cosine’]

#normalized metrics: rbf, laplacian




array([[ 1.        ,  0.36713288, -0.49438263, ...,  0.93817176,
         0.26485983,  0.72455268],
       [ 0.36713288,  1.        , -0.70683046, ...,  0.46766292,
         0.65104316, -0.10009633],
       [-0.49438263, -0.70683046,  1.        , ..., -0.35923055,
        -0.43796732,  0.0947128 ],
       ...,
       [ 0.93817176,  0.46766292, -0.35923055, ...,  1.        ,
         0.211382  ,  0.79763951],
       [ 0.26485983,  0.65104316, -0.43796732, ...,  0.211382  ,
         1.        , -0.36418777],
       [ 0.72455268, -0.10009633,  0.0947128 , ...,  0.79763951,
        -0.36418777,  1.        ]])

In [17]:
from math import sqrt



a = np.random.rand(3,2)

%time np.sum(a[0],axis=0)

%time [Parallel(n_jobs=8)(delayed(np.sum)(a[0],axis=0) for i in range(len(a)))]

CPU times: total: 0 ns
Wall time: 0 ns
CPU times: total: 31.2 ms
Wall time: 447 ms


[[0.8825171912593538, 0.8825171912593538, 0.8825171912593538]]